In [83]:
from random import randint
global num_of_iterations
num_of_iterations = 40

In [84]:
def hex_to_dec(hex_n):
    return int(hex_n, 16)

def dec_to_hex(n):
    return hex(n).replace("0x","").upper()

In [85]:
def mod_pow(x, y, m):
    # Returns (x^y) % p
    res = 1

    x = x % m
    while y > 0:
        if y & 1:
            res = (res * x) % m

        y = y >> 1
        x = (x * x) % m

    return res

# This function is called
# for all k trials. It returns
# false if n is composite and
# returns false if n is
# probably prime. d is an odd
# number such that d*2<sup>r</sup> = n-1
# for some r >= 1
def miller_rabin_test(d, n):
    # Pick a random number in [2..n-2]
    # Corner cases make sure that n > 4
    a = 2 + randint(1, n - 4)

    # Compute a^d % n
    x = mod_pow(a, d, n)

    if x == 1 or x == n - 1:
        return True

    while d != n - 1:
        x = (x * x) % n
        d *= 2

        if x == 1:
            return False
        if x == n - 1:
            return True

    # Return composite
    return False

# It returns false if n is
# composite and returns true if n
# is probably prime. k is an
# input parameter that determines
# accuracy level. Higher value of
# k indicates more accuracy.
def is_prime(n, k):
    # Corner cases
    if n <= 1 or n == 4:
        return False
    if n <= 3:
        return True

    # Find r such that n =
    # 2^d * r + 1 for some r >= 1
    d = n - 1
    while d % 2 == 0:
        d //= 2

    # Iterate given number of 'k' times
    for i in range(k):
        if not miller_rabin_test(d, n):
            return False

    return True

In [86]:
def generate_random_prime(range_min, range_max):
    global num_of_iterations
    while True:
        number = randint(range_min, range_max)
        if is_prime(number, num_of_iterations):
            return number

def generate_smaller_bigger(range_min, range_max):
    a = generate_random_prime(range_min, range_max)
    b = generate_random_prime(range_min, range_max)
    return (a, b) if a <= b else (b, a)

def generate_pairs(range_min, range_max):
    p_p1 = generate_smaller_bigger(range_min, range_max)
    q_q1 = generate_smaller_bigger(range_min, range_max)
    return (p_p1[0], q_q1[0]), (p_p1[1], q_q1[1])


In [87]:
def gcd_extended(a, m):
    if a == 0:
        return m, 0, 1
    gcd, x1, y1 = gcd_extended(m % a, a)
    x = y1 - (m // a) * x1
    y = x1
    return gcd, x, y


def inverse_modulo(a, m):
    gcd, x, y = gcd_extended(a, m)
    if gcd == 1:
        return (x % m + m) % m
    else:
        return -1

In [88]:
def generate_keys(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    while True:
        e = randint(2, phi - 1)
        if gcd_extended(e, phi)[0] == 1:
            d = inverse_modulo(e, phi)
            return e, n, d

In [89]:
class User:
    def __init__(self, name, p, q):
        self.name = name
        self.p = p
        self.q = q

        keys = generate_keys(p, q)
        self.e = keys[0]
        self.n = keys[1]
        self.d = keys[2]

    def regenerate_keys(self, receiver_n, range_min, range_max):
        """Regenerates keys if incompatible with receiver n"""
        while True:
            p = generate_random_prime(range_min, range_max)
            q = generate_random_prime(range_min, range_max)
            keys = generate_keys(p, q)
            if keys[1] <= receiver_n:
                self.p = p
                self.q = q
                self.e = keys[0]
                self.n = keys[1]
                self.d = keys[2]
                return

    def hex_regenerate_keys(self, receiver_n, range_min, range_max):
        """Regenerates keys if incompatible with receiver n"""
        self.regenerate_keys(hex_to_dec(receiver_n), range_min, range_max)

    def encrypt(self, m, e, n):
        c = mod_pow(m, e, n)
        return c

    def decrypt(self, c):
        m = mod_pow(c, self.d, self.n)
        return m

    def sign(self, m):
        s = mod_pow(m, self.d, self.n)
        return s

    def verify(self, m, s, e, n):
        m_check = mod_pow(s, e, n)
        return m == m_check

    def send_key(self, k, e, n):
        if self.n > n:
            print('Need to regenerate keys!')
            return
        k_enc = self.encrypt(k, e, n)
        s = self.sign(k)
        s_enc = self.encrypt(s, e, n)
        return k_enc, s_enc

    def receive_key(self, k_enc, s_enc, e, n):
        k = self.decrypt(k_enc)
        s = self.decrypt(s_enc)
        if self.verify(k, s, e, n):
            print('Success')
        else:
            print('Failure')

    def print_public(self):
        print(f'Modulus:\n{dec_to_hex(self.n)}\n\nE:\n{dec_to_hex(self.e)}')

    def hex_encrypt(self, m, e, n):
        return dec_to_hex(self.encrypt(hex_to_dec(m),
                                       hex_to_dec(e),
                                       hex_to_dec(n)))

    def hex_decrypt(self, m):
        return dec_to_hex(self.decrypt(hex_to_dec(m)))

    def hex_sign(self, m):
        return self.sign(hex_to_dec(m))

    def hex_verify(self, m, s, e, n):
        return self.verify(hex_to_dec(m),
                           hex_to_dec(s),
                           hex_to_dec(e),
                           hex_to_dec(n))

    def hex_send_key(self, key, e, n):
        k, s = self.send_key(hex_to_dec(key),
                             hex_to_dec(e),
                             hex_to_dec(n))
        return dec_to_hex(k), dec_to_hex(s)

    def hex_receive_key(self, k_enc, s_enc, e, n):
        self.receive_key(hex_to_dec(k_enc),
                         hex_to_dec(s_enc),
                         hex_to_dec(e),
                         hex_to_dec(n))

In [90]:
range_min = int('1'+'0'*255, 2)
range_max = int('1'*256, 2)
smaller,bigger = generate_pairs(range_min, range_max)

lil = User('Lil', smaller[0], smaller[1])
big = User('Big', bigger[0], bigger[1])

message = randint(1, 2 ** 30)
print(f'Generated message: {message}\n')

m_enc = lil.encrypt(message, big.e, big.n)
s = lil.sign(message)
print(f'Encrypted message from Lil to Big: {m_enc}')
print(f'Signature: {s}\n\n')


m_dec = big.decrypt(m_enc)
s_verify = big.verify(m_dec, s, lil.e, lil.n)
print(f'Big decrypted message: {m_dec}')
print(f'Signature verify: {s_verify}')

Generated message: 228280705

Encrypted message from Lil to Big: 8034899667930890294171721438826272079363972093456028660967473741193352922300675518791452410664435996657600364925338584525401073308687711833924417449424176
Signature: 1601100297349929380403311392854352747732847203372725719726531862502099462184121051141585738882467652167149569895275261177779158579239787382833416760506916


Big decrypted message: 228280705
Signature verify: True


In [91]:
k = randint(1, 2 ** 30)
print(f'Generated key: {k}\n')

k_enc, s_enc = lil.send_key(k, big.e, big.n)
big.receive_key(k_enc, s_enc, lil.e, lil.n)

Generated key: 360515213

Success


In [92]:
me = User('me', generate_random_prime(range_min, range_max), generate_random_prime(range_min, range_max))
me.print_public()

Modulus:
9F3FFBB3C9349849A27A2A7EA9E52B288ACFA563F532B615EE3010B1227DC9350D2082209C087F2E65FF8BA2D742970ABD1B92D6D017DCC0AE3AF7F9E1D1312F

E:
5747699930490AC4D58122D4D18FC73C943FEAC835DEE93488CF3EEC410C6EA76FE301938049547113556C3C863E99B7F104E1EF755831074BB3D90A0B72DCC5


In [93]:
# Server modulus here
n_hex = '94429BC353A2492FF546D7C266F6213D12EAB30228C49BE841775AD71D359200D026FEF407DC3C9DFBE0FCC7CC0215B0B4547D3EC21A271B08F35A6B2020E5CD'
e_hex = '10001'

key = 'ABCD'

me.hex_regenerate_keys(n_hex, range_min, range_max)
me.print_public()
me.hex_send_key(key, e_hex, n_hex)

Modulus:
78A78F33494907A41FE5158C249DD9EFD60D7EB4C68959FD792882157FF23C20431BBF37B13BB42461F10BC79F213B6F5738EEE94B62E66C4E33FDE090CFBC99

E:
455FCE0604E613E2B2014D09F597BFAE93647F388442AEB7590A5A1744F3810CEF2DB93B6FAFDEF4157FA10FDC576C2275497F3169D6A46D06149B202E31176F


('56683DD20F8D3410BE95BE9581B4D9A9A5A1E95535C020B7024E3FECF6DED4ED3DF289625946B08FB54D8A89FE5E01ACDAC0ECE6665F66793BF878315DB9BC9C',
 '26981109BA9E4B9B79D8100A35E6EE5795D0DE7C15DD7F9107A364A48690493A2FA625B512AE0F12EDFBF96CE047C2D6E9338AACBF0B672205F903AD863D5936')